In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.7/405.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.7/915.7 MB 682.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.3/188.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.

In [1]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"

max_seq_length = 1024  # Choose sequence length
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [2]:
import json

# Load your existing JSON
with open("sql json.json", "r", encoding="utf-8") as f:
    data = json.load(f)

examples = data["examples"]

# Write training-ready JSONL
with open("hr_dataset_train.jsonl", "w", encoding="utf-8") as f:
    for ex in examples:
        record = {
            "messages": [
                {"role": "system", "content": ex["system"]},
                {"role": "user", "content": ex["user"]},
                {"role": "assistant", "content": ex["assistant"]}
            ]
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ Converted {len(examples)} samples into training JSONL")

✅ Converted 31 samples into training JSONL


In [3]:
from datasets import load_dataset

dataset = load_dataset(
    "json",
    data_files="hr_dataset_train.jsonl",
    split="train"
)

print(dataset[0])

Generating train split: 0 examples [00:00, ? examples/s]

{'messages': [{'role': 'system', 'content': 'You are an HR SQL assistant. Convert HR-related questions into SQL queries.'}, {'role': 'user', 'content': 'Show all employees'}, {'role': 'assistant', 'content': 'SELECT * FROM employees;'}]}


In [4]:
def format_for_training(example):
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

dataset = dataset.map(format_for_training)

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r=64,  # LoRA rank - higher = more capacity, more memory
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=128,  # LoRA scaling factor (usually 2x rank)
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",     # Supports any, but = "none" is optimized
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized version
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None, # LoftQ
)

Unsloth 2026.1.4 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments

# Training arguments optimized for Unsloth
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=2,  # Effective batch size = 8
        warmup_steps=10,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=25,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/31 [00:00<?, ? examples/s]

In [7]:
from unsloth import FastLanguageModel
import torch

FastLanguageModel.for_inference(model)

# ✅ Use SAME system prompt as training
messages = [
    {
        "role": "system",
        "content": "You are an HR SQL assistant. Convert HR-related questions into ONLY SQL queries."
    },
    {
        "role": "user",
        "content": "Number of absent today"
    }
]

# Apply chat template (DO NOT hardcode tokens)
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

attention_mask = inputs.ne(tokenizer.pad_token_id).long()

# Generate
with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        temperature=0.0,      # 🔥 Lower = deterministic SQL
        do_sample=False,      # 🔥 SQL should be exact
        top_p=1.0,
        use_cache=True,
    )

# Decode
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# ✂️ Extract assistant response only
assistant_response = decoded.split("assistant")[-1].strip()

print(assistant_response)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


To get the number of absent employees today, you can use the following SQL query:

```sql
SELECT COUNT(*) AS absent_count
FROM employees
WHERE absence_date IS NOT NULL AND absence_date!= 'null';
```

This query will return the total number of absent employees today. 

However, if you want to get the count of absent employees for a specific date, you can use the following query:

```sql
SELECT COUNT(*) AS absent_count
FROM employees
WHERE absence_date = '2022-01-01' OR absence_date = '2022-01-02' OR absence_date = '2022-01-03';
```

This query will return the count of absent employees for the specified dates.


In [8]:
model.save_pretrained("hr_sql_hf", safe_serialization = True)
tokenizer.save_pretrained("hr_sql_hf")

('hr_sql_hf/tokenizer_config.json',
 'hr_sql_hf/special_tokens_map.json',
 'hr_sql_hf/chat_template.jinja',
 'hr_sql_hf/tokenizer.json')

In [9]:
from huggingface_hub import login, whoami

login()

whoami()

{'type': 'user',
 'id': '697899cb4d5c19f2bed9f2d0',
 'name': 'Sivadharany',
 'fullname': 'Sivadharany N',
 'isPro': False,
 'avatarUrl': '/avatars/2d6fab07a1384bd4a04499f61e768448.svg',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'Llama token',
   'role': 'fineGrained',
   'createdAt': '2026-01-27T11:19:53.617Z',
   'fineGrained': {'canReadGatedRepos': True,
    'global': [],
    'scoped': [{'entity': {'_id': '66eaedefece5ee215637cc82',
       'type': 'model',
       'name': 'meta-llama/Llama-3.2-1B-Instruct'},
      'permissions': ['repo.content.read', 'discussion.write', 'repo.write']},
     {'entity': {'_id': '697899cb4d5c19f2bed9f2d0',
       'type': 'user',
       'name': 'Sivadharany'},
      'permissions': ['repo.content.read',
       'repo.write',
       'inference.serverless.write',
       'inference.endpoints.infer.write',
       'inference.endpoints.write',
       'user.webhooks.read',
       'user.webhooks.write',
       'collection.read'

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_id = "meta-llama/Llama-3.2-1B-Instruct"  # your base model
adapter_path = "hr_sql_hf"

tokenizer = AutoTokenizer.from_pretrained(base_model_id)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype="auto",
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, adapter_path)

# 🔥 THIS IS THE CRITICAL STEP
model = model.merge_and_unload()

`torch_dtype` is deprecated! Use `dtype` instead!


In [11]:
model.save_pretrained("hr_sql_merged", safe_serialization=True)
tokenizer.save_pretrained("hr_sql_merged")

('hr_sql_merged/tokenizer_config.json',
 'hr_sql_merged/special_tokens_map.json',
 'hr_sql_merged/chat_template.jinja',
 'hr_sql_merged/tokenizer.json')

In [12]:
import shutil
from google.colab import files

# Replace 'my_folder' with the folder name you want to download
shutil.make_archive("hr_sql_merged", 'zip', "hr_sql_merged")

# This creates my_folder.zip in your Colab environment
files.download("hr_sql_merged.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>